# Notebook for analytics on the result of the republic referendum

### Libraries

In [24]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Image

In [5]:
filepath = '1999_referenda_output/republic_referendum_by_electorate_by_polling_place.csv'
df_results = pd.read_csv(
    filepath
)

display(df_results.head(3))

,index,state,electorate,polling_place,polling_place_raw,yes_or_no,yes_n,yes_p,no_n,no_p,formal_n,formal_p,informal_n,informal_p,total_n,total_p
0,0,SA,Adelaide,Adelaide,Adelaide (Adelaide),Yes,282,0.6144,177,0.3856,459,0.9871,6,0.0129,465,0.0057
1,1,SA,Adelaide,Adelaide East,Adelaide East,Yes,465,0.6700,229,0.3300,694,0.9914,6,0.0086,700,0.0086
2,2,SA,Adelaide,Adelaide Hospital,Adelaide Hospital,Yes,187,0.6172,116,0.3828,303,0.9806,6,0.0194,309,0.0038


In [6]:
filepath = '1999_referenda_output/republic_referendum_by_polling_place.csv'
df_results_by_pp = pd.read_csv(
    filepath
)

display(df_results_by_pp.head(3))

,state,polling_place,yes_n,no_n,formal_n,informal_n,total_n,yes_p,no_p,formal_p,informal_p
0,ACT,Ainslie,1372,500,1872,24,1896,0.7329,0.2671,0.9873,0.0127
1,ACT,Ainslie North,1608,749,2357,29,2386,0.6822,0.3178,0.9878,0.0122
2,ACT,Aranda,2200,787,2987,21,3008,0.7365,0.2635,0.9930,0.0070


In [7]:
filepath = '1999_referenda_output/polling_places_geocoded.csv'
df_pp = pd.read_csv(
    filepath
)

display(df_pp.head(3))

,state,polling_place,premises,address,suburb,postcode,wheelchair_access,match_source,match_type,latitude,longitude
0,ACT,Bonython,Bonython Primary School,Hurtle Ave,BONYTHON,2905.0,F,2007 Polling Places,"Match 01 - state, premises, postcode",-35.4318,149.083
1,ACT,Calwell,Calwell High School,Casey Cres,CALWELL,2905.0,F,2007 Polling Places,"Match 01 - state, premises, postcode",-35.4406,149.116
2,ACT,Canberra Hospital,The Canberra Hospital,Blding 2 Level 3 Yamba Dr,GARRAN,2605.0,F,2007 Polling Places,"Match 01 - state, premises, postcode",-35.3453,149.100


### Result by state/territory

* The ACT comfortably voted in favour, and was the only jurisdiction to do so
* Victoria was very narrowly opposed
* Queensland was almost 2 to 1 against

In [8]:
r = df_results[['state','yes_n','formal_n']].groupby('state').sum()
r['yes_p'] = round(r['yes_n']/r['formal_n'],4)
display(r.sort_values(['yes_p'],ascending=False))

,yes_n,formal_n,yes_p
state,,,
ACT,127211,201061,0.6327
VIC,1489536,2988674,0.4984
NT,44391,91028,0.4877
NSW,1817380,3913942,0.4643
SA,425869,977444,0.4357
WA,458306,1104826,0.4148
TAS,126271,312784,0.4037
QLD,784060,2094052,0.3744


### The inner cities were the most strongly in favour

The top 5 seats by proportion in favour represent two inner Melbourne and two inner Sydney seats, as well as one of the two Canberra seats

In [11]:
r = df_results[['electorate','yes_n','formal_n']].groupby('electorate').sum()
r['yes_p'] = round(r['yes_n']/r['formal_n'],4)
display(r.sort_values(['yes_p'],ascending=False).head(5))

,yes_n,formal_n,yes_p
electorate,,,
Melbourne,59994,84598,0.7092
Sydney,56921,83894,0.6785
Melbourne Ports,51520,78183,0.6590
Grayndler,51774,79929,0.6477
Fraser,64636,100266,0.6446


### Rural Australia was the most strongly opposed

The top 5 seats by proportion opposed are all large rural seats, four in Queensland and one (Gwydir) in NSW.

In [12]:
r = df_results[['electorate','yes_n','formal_n']].groupby('electorate').sum()
r['yes_p'] = round(r['yes_n']/r['formal_n'],4)
display(r.sort_values(['yes_p'],ascending=True).head(5))

,yes_n,formal_n,yes_p
electorate,,,
Maranoa,17944,78554,0.2284
Blair,18078,71299,0.2536
Wide Bay,19052,74205,0.2567
Groom,21406,78067,0.2742
Gwydir,19274,69355,0.2779


### How predictiveis the geographic size of seat on support for the referendum?

#### Prepare data

In [23]:
# import geographic size of seats
filepath = '1999_referenda/electorate_boundaries/boundaries_republic_referendum_aus.csv'

df_area = pd.read_csv(
    filepath,
    skiprows = 1,
    names = ['electorate','area_sqkm']
)

# make df grouped by electorate

df_by_electorate = df_results[['electorate','yes_n','formal_n']].groupby('electorate').sum()
df_by_electorate['yes_p'] = round(r['yes_n']/r['formal_n'],4)

df_by_electorate = df_by_electorate.reset_index()

# merge in area
df_by_electorate = pd.merge(df_by_electorate, df_area, on='electorate', how='left')

display(df_by_electorate.head(5))

,electorate,yes_n,formal_n,yes_p,area_sqkm
0,Adelaide,45580,80832,0.5639,80.80
1,Aston,43210,83822,0.5155,128.10
2,Ballarat,32784,80129,0.4091,11128.95
3,Banks,34719,75875,0.4576,61.70
4,Barker,26709,82364,0.3243,67077.85


#### create data for scatterplot

In [29]:
from plotly.offline import *
import plotly.offline as py
import plotly.plotly as pyonline
import plotly.graph_objs as go
init_notebook_mode(connected=True) # render plotly charts in the notebook on the fly

In [36]:
series = go.Scatter(
    y = df_by_electorate['yes_p'],
    x = df_by_electorate['area_sqkm'],
    name = '% Yes',
    mode = 'markers',
    text = df_by_electorate['electorate'],
    marker = dict (
        size = 10,
        opacity = 0.7
    )
)

xaxis=dict(
        title = 'Size of Electorate, SqKm',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
)

yaxis = dict(
        title = '% Support',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
)

title = '1999 Republic Referendum - % Support vs. Size of Electorate'

titlefont = dict(
        family='Open Sans',
        size=22
)

layout = go.Layout(
    title = title,
    titlefont = titlefont,
    xaxis = xaxis,
    yaxis = yaxis
)

data = [series]

figure01 = go.Figure(data=data, layout=layout)

In [37]:
py.iplot(figure01, filename='figure01')
# pyonline.image.ishow(figure01, width=1500, height=750)

### Same chart, using a log scale ofr electorate size

In [46]:
xaxis=dict(
        title = 'Log of size of Electorate, SqKm',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        type='log'
)

title = '1999 Republic Referendum - % Support vs. log(size) of Electorate'

layout = go.Layout(
    title = title,
    titlefont = titlefont,
    xaxis = xaxis,
    yaxis = yaxis
)

figure02 = go.Figure(data=data, layout=layout)

In [47]:
py.iplot(figure02, filename='figure02')
# pyonline.image.ishow(figure02, width=1500, height=750)